In [1]:
import numpy as np
import pandas as pd
import pymysql as sql
from sklearn.datasets import load_iris

In [2]:
raw = load_iris()
iris = pd.DataFrame(raw.data, columns=raw.feature_names)

var_names = [val.replace(' ', '_').replace('_(cm)', '') for val in list(iris.columns)]

In [3]:
#test for connection
con = sql.connect('localhost', 'Iris_test', 
    'Iris_12345678', 'Iris_db')

with con:
    
    cur = con.cursor()
    cur.execute("SELECT VERSION()")

    version = cur.fetchone()
    
    print("Database version: {}".format(version[0]))
    
con.close()

Database version: 5.7.25-0ubuntu0.18.04.2


In [4]:
#create table 'iris'
#do not run this gain!!
#recommend to save the login info. in a '.py' file
#use .format instead of puting var_names in the cur.execute func.. In the case the command will be like ....'sepal_length'... , and cannot be executed

con = sql.connect(host='localhost',
        user='Iris_test',
        password='Iris_12345678',
        db='Iris_db',
        charset='utf8mb4',
        cursorclass=sql.cursors.DictCursor)

with con:
    
    cur = con.cursor()
    
    command =  '''CREATE TABLE Iris(id INT UNSIGNED NOT NULL AUTO_INCREMENT,
                           {0[0]} DECIMAL(18, 2),
                           {0[1]} DECIMAL(18, 2),
                           {0[2]} DECIMAL(18, 2),
                           {0[3]} DECIMAL(18, 2),
                           PRIMARY KEY (id));'''.format(var_names)
    
    cur.execute(command)
    
    con.commit()

con.close()

In [5]:
#reconstruct the df and change dtype as np.float64 is not supported by cur.excutemany()
data_remake = [[float(iris.values[i,j]) for j in range(4)] for i in range(len(iris))]

In [6]:
#try insert many times at once
#do not run this again
#cannot use %.2f to reflect data in the query!!
#only %s should be used

try:
    con = sql.connect(host='localhost',
        user='Iris_test',
        password='Iris_12345678',
        db='Iris_db',
        charset='utf8mb4',
        cursorclass=sql.cursors.DictCursor)
    
    sql_query = '''INSERT INTO Iris ({0[0]}, {0[1]}, {0[2]}, {0[3]}) VALUES(%s, %s, %s, %s);'''.format(var_names)
    
    cur = con.cursor()
    cur.executemany(sql_query, data_remake)
    
    con.commit()
    print (cur.rowcount, 'Record(s) inserted successfully into python_users table')

except sql.err.Error as err:
    print("Failed inserting record into python_users table. {}".format(err))
    
finally:
    try:
        con.close()
        print('connection closed sucessfully')
    except sql.connections.err.Error as err:
        print('connection might have been closed, or cannot be closed...')
        print('for detail, see: ', err)


150 Record(s) inserted successfully into python_users table
connection closed sucessfully


In [7]:
#select table
#commit not needed

try:
    con = sql.connect(host='localhost',
        user='Iris_test',
        password='Iris_12345678',
        db='Iris_db',
        charset='utf8mb4',
        cursorclass=sql.cursors.DictCursor)
    
    sql_query = '''SELECT id, {0[0]}, {0[1]}, {0[2]}, {0[3]} FROM Iris;'''.format(var_names)
    
    cur = con.cursor()
    cur.execute(sql_query)
    
    output = cur.fetchall()

except sql.err.Error as err:
    print("Something bad happened during query {}".format(err))
    
    
finally:
    con.close()
    print('connection closed sucessfully')

connection closed sucessfully


In [8]:
#output is a dictionary, and can be converted to dataframe by:
#some data seem to be string, we need to parse them to be in appropriate formats
df = pd.DataFrame(output)
df.dtypes

id               int64
petal_length    object
petal_width     object
sepal_length    object
sepal_width     object
dtype: object

In [9]:
df.loc[:,'petal_length':'sepal_width'] = df.loc[:,'petal_length':'sepal_width'].astype('float64')
df.head(10)

,id,petal_length,petal_width,sepal_length,sepal_width
0,1,1.4,0.2,5.1,3.5
1,2,1.4,0.2,4.9,3.0
2,3,1.3,0.2,4.7,3.2
3,4,1.5,0.2,4.6,3.1
4,5,1.4,0.2,5.0,3.6
5,6,1.7,0.4,5.4,3.9
6,7,1.4,0.3,4.6,3.4
7,8,1.5,0.2,5.0,3.4
8,9,1.4,0.2,4.4,2.9
9,10,1.5,0.1,4.9,3.1


In [10]:
#drop table Iris

try:
    con = sql.connect(host='localhost',
        user='Iris_test',
        password='Iris_12345678',
        db='Iris_db',
        charset='utf8mb4',
        cursorclass=sql.cursors.DictCursor)
    
    sql_query = 'DROP TABLE Iris;'
    
    cur = con.cursor()
    cur.execute(sql_query)
    
    con.commit()
    
    print('Table Dropped sucessfully')

except sql.err.Error as err:
    print("Something bad happened during query {}".format(err))
    
    
finally:
    con.close()
    print('connection closed sucessfully')

Table Dropped sucessfully
connection closed sucessfully
